In [1]:
import pandas as pd
from itertools import chain
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient
from collections import Counter

In [2]:
data_dir = '../synonyms_from_category/data/'
dt = '20190130'

In [3]:
df_synonyms = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/candidate_synonyms_final_190130.tsv', sep='\t', names=['word', 'synonyms'])


In [4]:
df_users = pd.read_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/cate_WE_user_dict_190130.txt', names=['word'])

In [5]:
df = df_synonyms.append(df_users, ignore_index=True)

/home/admin-/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [6]:
df = df.sort_values(by=['word']).reset_index(drop=True)
df.synonyms = df.synonyms.astype('str')
df.synonyms = df.synonyms.fillna(value=0)
type(df.synonyms[1])

In [11]:
df['output'] = None


def word_and_synonyms(row):
    temp_str = row['word']
    if row['synonyms'] != 'nan':
        temp_str = temp_str + " " + row['synonyms']
        temp_list = temp_str.split(" ")
    else:
        temp_list = [temp_str]
    row['output'] = temp_list
    return row


df = df.apply(lambda row: word_and_synonyms(row), axis=1)

In [12]:
df.output.sample(20)

222                                        [동양화]
191                                       [닭가슴살]
158                                 [네일컬러 네일 컬러]
320                                        [매직기]
1127                                       [지퍼백]
1281                                       [키지갑]
332                            [머신, machine, 머쉰]
543     [브래지어, 브라자, bra, 브라, 브래이지어, 브레이지어, 브레지어]
977                                       [운동기구]
291                                   [리얼, real]
288                                        [리무버]
78                                        [공압공구]
268                                       [런닝머신]
1456                   [헤어, hair, 모발, 머리카락, 머리칼]
613                                        [생리대]
32                     [가죽, leather, 래더, 레더, 피혁]
358                                        [모기장]
127                             [나이키, nike, 니이키]
874                                        [양념장]
848                                        [안경줄]
Name: output, dtype:

In [14]:
list_output = df.output.tolist()
list_output = [_.lower() for _ in chain(*list_output)]

In [16]:
print(len(pd.Series(list_output)), len(pd.Series(list_output).unique()))

2841 2722


In [18]:
pd.Series(list_output).to_csv('/home/admin-/PycharmProjects/playground/synonyms_from_category/data/freq_user_dict_'+dt+'.tsv', index=False)